**Text pre-processing**

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # for regex
import sys
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
#import matplotlib.pyplot as plt and seaborn for visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
# BOW
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,precision_score,recall_score, confusion_matrix
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 6

sentiments_df = pd.read_csv('Tweets.csv')
print(sentiments_df.shape)
sentiments_df.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
# exploratory analysis

from wordcloud import WordCloud

text = " ".join(review for review in sentiments_df.text)


wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="black", stopwords=stopwords.words("english")).generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show();

AttributeError: ignored

In [ ]:
# removing HTML Tags
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

sentiments_df.text = sentiments_df.text.apply(clean)
sentiments_df.text[0]


In [ ]:
# removing special characters

def is_special1(text):
    rem = ''
    for i in text:
        if i.isalnum()==False and i != " ":  #checks if character is alphanumeric
            text=text.replace(i,'')
            
    return text
sentiments_df.review = sentiments_df.review.apply(is_special1)
sentiments_df.review[0]

In [ ]:
# converting to lower case
def to_lower(text):
    return text.lower()

sentiments_df.review = sentiments_df.review.apply(to_lower)
sentiments_df.review[0]

In [ ]:
# These are articles, prepositions, or conjunctions like “the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc 
# using 'stopword' from nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

sentiments_df.review = sentiments_df.review.apply(rem_stopwords)
sentiments_df.review[0]

In [ ]:
# stemming: converting words into their stem words e.g plays, playing to 'play'
def stem_txt(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

sentiments_df.review = sentiments_df.review.apply(stem_txt)
sentiments_df.review[0]

In [ ]:
#count vectorizer only count the number of times a word appears in the document and ends up ignoring rare words that could have helped in processing data efficiently
# TfidfVectorizer consider the overall weightage of a word

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer


X= np.array(sentiments_df.iloc[:,0].values)
y = np.array(sentiments_df.sentiment.values)

tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
text_count_matrix = tfidf.fit_transform(sentiments_df.review)



In [ ]:
# split the data
x_train, x_test, y_train, y_test = train_test_split(text_count_matrix, y, test_size=0.30, random_state=2)

In [ ]:
# using the 2 types of Naive bayes algorithm
# Note:TfdifVectorized data cannot be implemented by Gaussian NB
m=MultinomialNB(alpha=1.0,fit_prior=True)
b=BernoulliNB(alpha=1.0,fit_prior=True)


In [ ]:
b.fit(x_train,y_train)
m.fit(x_train,y_train)
yup=b.predict(x_test)
yup2=m.predict(x_test)
print("Multinomial accuracy = ",accuracy_score(y_test,yup2))
print("Bernoulli accuracy = ",accuracy_score(y_test,yup))

print("Multinomial precision= ",precision_score(y_test,yup2))
print("Bernoulli precision= ",precision_score(y_test,yup))

print("Multinomial recall= ",recall_score(y_test,yup2))
print("Bernoulli recall= ",recall_score(y_test,yup))

In [ ]:
# Multinomial Naive Bayes
cnf_matrix = confusion_matrix(y_test, yup2)
cnf_matrix

In [ ]:

class_names = ["negative", "positive"]
fig,ax = plt.subplots()


sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names)
ax.xaxis.set_label_position('top')
plt.tight_layout()
plt.ylabel('Actual sentiment')
plt.xlabel('Predicted sentiment');